<div style="padding: 20px; border-radius: 10px; border: 2px solid #3A506B; background-color: #F1F6F9; color: #1C2541;">
    <h1 style="font-family: Arial, sans-serif; color: #3A506B; font-size: 2.5em; text-align: center; line-height: 1.2;">
        <span style="font-weight: bold;">Intro to Deep Learning</span>
    </h1>
    <h2 style="font-family: Arial, sans-serif; color: #1C2541; text-align: center; font-size: 1.5em; font-weight: normal;">
        2024-2025
    </h2>
    <h2 style="font-family: Arial, sans-serif; color: #1C2541; text-align: center; font-size: 1.5em; font-weight: normal;">
        Course ID: 05714182
    </h2>
    <h2 style="font-family: Arial, sans-serif; color: #1C2541; text-align: center; font-size: 1.5em; font-weight: normal;">
        ----------------------------------
    </h2>
    <h2 style="font-family: Arial, sans-serif; color: #1C2541; text-align: center; font-size: 1.5em; font-weight: normal;">
        Yahlly Schein
    </h2>
</div>


<div style="padding: 20px; border-radius: 10px; border: 2px solid #3A506B; background-color: #FF8C00; color: #1C2541;">
    <h2 style="font-family: Arial, sans-serif; color: #1C2541; text-align: center; font-size: 1.5em; font-weight: normal;">
        HW 2
    </h2>
</div>

In this assignment, you will explore the power of transfer learning by fine-tuning pre-trained deep learning models for image classification. 

This assignment challenges you to understand how to utilize state-of-the-art models effectively, adapt them to new domains, and make design choices that align with real-world constraints. and yes, this assignment is intentionally more complex than the previous one, as it is designed to encourage you to better understand the material and dive deeper into its underlying concepts.

While solving this assingment you will see how training process look in "real life". Expect to see some wierd results in the graphs (due to possible mistakes you are going to do) and put some effort into understanding what is going on and fix these issues. This is where you learn. We have talked about everything in class.


Read the marked text in this notebook - it will guide you and help you a lot! 

**By the end, you’ll have practical experience with one of the most impactful techniques in modern deep learning!**

<div style="padding: 20px; border-radius: 10px; border: 2px solid #3A506B; background-color: #FF3C00; color: #1C2541;">
    <h2 style="font-family: Arial, sans-serif; color: #1b1541; text-align: center; font-size: 1.5em; font-weight: normal;">
        ASK YOUR QUESTIONS IN THE DEDICATED Q & A FORUM IN THE MOODLE
    </h2>
</div>

# Transfer Learning with Pytorch

In this excercise, you will implement a 2-class classification neural network, using transfer learning from a pre-trained resnet18 network. A pre-trained model is a model that was trained on a large benchmark dataset to solve a problem similar to the one that we want to solve (but not identical). The pre-trained network could have been trained on a different dataset and sometimes also on a different task. The important thing is that it would be a related task and a realted dataset. This way, we can leverage previous learneinng to solve the new problem (instead of training the model from scratch). The ResNet18 model was pre-trained on the [ImageNet](http://image-net.org/index) dataset (a large dataset consisting of 1.4M images and 1000 classes). 

Transfer learning can be used in two ways:
1. Feature extractor: in this way, we remove top layers, freeze the weights of all remaining layers, add a new fully connected layers on top of the pretrained model. Then. we train only these new layers to solve the new task.
2. Fine tuning: Here we unfreezing the entire model (or part of it), add a new fully connected layers on top of the pre-trained model (as before) and re-train the entire model on the new data with a very low learning rate. We can use either end-to-end re-training, or re-train only part of the network.

### CIFAR-10 Dataset:

In this assignment, you will need to perform transfer learning using the ResNet18 network in order to classify and discern between images of cats and dogs in a different dataset: [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) 



# Part 1- Feature Extractor

In this part of the exercise, you need to classify images of 'Cat' and 'Dog' by using transfer learning with the feature extractor approach and a pre-trained ResNet18 network.

The pre-trained model is built out of two main parts, the features and the classifier. 
- The features part is a stack of convolutional layers. You will need to extract the image representation that will be fed into a classifier. 
- For the classifier, please use a *single fully-connected layer*. 


In order to make things efficient, start by using the ResNet18 network in order to extract image represetnations for the images in CIFAR-10 dataset. Please make sure to this only once (please do not extract the image representations each time it is needed in the training phase)! You can store the resulting image representations as feature vectors and train the classifier using these vectors. 
At test time, you can merge both networks in order to achive a single end-to-end classifier.  
Note: This is different from what you saw in class.

At the test time, you can merge the feature extraction netowork with the new prediction head, in order to recive a "full solution".


**Let's first import all the packages that you will need during this part of assignment:**

Feel free to import more packages if you want to

In [1]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, models, transforms

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import time

import sklearn
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score, roc_curve, auc

ModuleNotFoundError: No module named 'torchvision'

The ResNet18 network requires images of at least 200x200 pixels. However, the CIFAR-10 dataset consists of small images of 32x32 pixels. Hence, you will need to reshape the images before we use them into the ResNet18 network.

Additionally, you will need to match the normalization used while training the ResNet18: each color channel should be normalized separately. 

The channel means are [0.485, 0.456, 0.406] and the standard deviations are [0.229, 0.224, 0.225].

**Use the transforms library from  torchvision and definethese transformations:**

In [ ]:
# Specify transforms using torchvision.transforms as transforms library
from torchvision import transforms

# Specify transformations
transform = transforms.Compose([
    # Resize images to 200x200 pixels
    transforms.Resize((200, 200)),
    
    # Convert images to PyTorch tensors
    transforms.ToTensor(),
    
    # Normalize images using ResNet18's pretraining statistics
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # Mean for each channel
        std=[0.229, 0.224, 0.225]    # Standard deviation for each channel
    )
])

# Display the transform pipeline
print(transform)

#### Load CIFAR10 dataset and transform dataset from torchvision:

In [ ]:
# Load the CIFAR-10 dataset and apply the transformations
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Verify the data shape and transformations
print(f"Train Dataset: {len(train_dataset)} samples")
print(f"Test Dataset: {len(test_dataset)} samples")
print(f"Example data shape: {train_dataset[0][0].shape}")  # Example image tensor shape

**Note that the CIFAR10 data set includes 10 labels. Please extract from the entire dataset, only the images with 'Cat' or 'Dog' labels.**

In order to make the training process faster, you will need to **trim** the train set to 800 training images of each label (800 dogs and 800 cats). 
The test set should be trimmed to 250 images of cats and 250 images of dogs.

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# Load the full CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
full_test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Map label names to indices
label_map = {
    'cat': 3,  # Label index for 'cat'
    'dog': 5   # Label index for 'dog'
}

# Function to extract specific labels
def extract_and_trim(dataset, target_labels, num_samples_per_label):
    indices = []
    counts = {label: 0 for label in target_labels}

    # Iterate through the dataset and collect indices of desired labels
    for idx, (_, label) in enumerate(dataset):
        if label in target_labels and counts[label] < num_samples_per_label:
            indices.append(idx)
            counts[label] += 1
        # Stop when we have enough samples for all labels
        if all(count >= num_samples_per_label for count in counts.values()):
            break

    # Return a subset of the dataset with the selected indices
    return Subset(dataset, indices)

# Extract and trim the datasets
train_dataset = extract_and_trim(full_train_dataset, [label_map['cat'], label_map['dog']], 800)
test_dataset = extract_and_trim(full_test_dataset, [label_map['cat'], label_map['dog']], 250)

# Create DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Verify the sizes of the datasets
print(f"Train dataset size: {len(train_dataset)} samples")
print(f"Test dataset size: {len(test_dataset)} samples")


**Download the ResNet18 Network from torchvision.models:**

In [ ]:
# Download the pre-trained ResNet18 model
resnet18 = models.resnet18(pretrained=True)

# Print the model architecture to verify
print(resnet18)

#### Now, use the pre-trained Resnet18 to extract feautres from the CIFAR10 dataset and feed your classifier with these feature vectors. To do that, you need to cut the head of the pre-trained ResNet18 model, then feed the model with the CIFAR10 data sets (train and test).  The resulting image representations should be "flattened" and used as feature vectors:

In [ ]:
# Load pre-trained ResNet18
resnet18 = models.resnet18(pretrained=True)

# Remove the fully connected (fc) layer to use ResNet18 as a feature extractor
feature_extractor = nn.Sequential(*list(resnet18.children())[:-1])
feature_extractor.eval()  # Set to evaluation mode

# Function to extract features
def extract_features(data_loader, feature_extractor, device='cpu'):
    features = []
    labels = []
    feature_extractor.to(device)  # Move model to device
    with torch.no_grad():  # Disable gradient computation
        for images, targets in tqdm(data_loader):
            images = images.to(device)
            # Pass images through the feature extractor
            outputs = feature_extractor(images)
            # Flatten the feature maps to vectors
            outputs = outputs.view(outputs.size(0), -1)
            features.append(outputs.cpu())
            labels.append(targets)
    return torch.cat(features), torch.cat(labels)

# Device setup (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create DataLoaders for trimmed datasets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Extract features and labels for training and testing datasets
train_features, train_labels = extract_features(train_loader, feature_extractor, device)
test_features, test_labels = extract_features(test_loader, feature_extractor, device)

# Verify shapes of extracted features
print(f"Train features shape: {train_features.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Test features shape: {test_features.shape}")
print(f"Test labels shape: {test_labels.shape}")

# Logistic Regression with scikit-learn library:

#### Here, you need to use a logistic regession model from scikit-learn and train it with the data you prepared above. You need to calculate and report the accuracy and AUC for the train and test sets and explain your results.

In [ ]:
### START CODE HERE ###


# Logistic Regression with pytorch:

#### Next, you need to use a logistic regession using pythourch as follows:


1. Explain the choice of architecture in the head of the neural network.
2. Plot training loss per epoch.
3. Plot AUC per epoch for train and validation sets.
4. Explain your results and findings.
5. Does adding one or two hidden layers affect the generalization capabilities of the model?

#### Define the model:

In [ ]:
### START CODE HERE ###


#### Explain your choice:

In [ ]:
### Explain HERE ###

#### Training and validation:

In [ ]:
### START CODE HERE ###


#### Visualizing the plots:

In [ ]:
### START CODE HERE ###


#### Explain the results:

In [ ]:
### Explain HERE ###

**Here you should merge the feature extractor network layers with the PyTorch logistic regression classifier.**

In [ ]:
### START CODE HERE ###


# Comparing  ResNet18 with ResNet34:

#### Download the ResNet34 Network from torchvision.models. 

##### Then, repeat the steps described above in order to compare between these ResNet18 and ResNet43. 
##### Explain your findings and results!

In [ ]:
### START CODE HERE ###


# Logistic Regression with scikit-learn library:

In [ ]:
### START CODE HERE ###


# Logistic Regression with pytorch for ResNet34:

#### Define the model:

In [ ]:
### START CODE HERE ###


#### Training and validation:

In [ ]:
### START CODE HERE ###


#### Visualizing the plots:

In [ ]:
### START CODE HERE ###


**Here you should merge the feature extractor network layers with the PyTorch logistic regression classifier.**

In [ ]:
### START CODE HERE ###


#### Explain the findings and results:

In [ ]:
### Explain HERE ###

# Part 2- Fine-tuning

In the feature extraction experiment above you were training the classifier on the top of the pre-trained model. 

In this part, you need to train (or "fine-tune") the original pre-trained Resnet18 model (or part of them) in an attempts to improve your results. You can determine the number of convolution layers you intend to re-train in order to avoid overfitting and hopefully achive better generalization. 

Note that the fine-tuning should only be attempted after you have trained the top-level classifier with a pre-trained model.

1. Plot Loss and Accuracy v.s Epoch for train and validation sets:

#### Define the model (use the merged feature extractor with the PyTorch logistic regression classifier)

In [ ]:
### START CODE HERE ###


#### Training and validation:

In [ ]:
### START CODE HERE ###


#### Visualizing the plots:

In [ ]:
### START CODE HERE ###


### Compare your results between the two approaches. Is there any improvement? 
### Explain your findings...


# Part 3-Open questions


1. Mention 5 concepts/function/blocks of code from the exercises that you used while solving the assignment.

2. Mention 3 comcepts/ code parts/ anything that a LLM (ChatGPT, Claude...) helped you with:

3. How practical and meaningfull assingment did you find this one on a scale from 1 (very bad) to 10 (OMG WOW)?

4. Which concepts or techniques became clearer to you while working on this assignment?